In [24]:
class LambdaLayer(nn.Module):
    def __init__(self, dim, dim_k, dim_out, heads):
        super(LambdaLayer, self).__init__()

        assert(dim_out % heads) == 0
        heads = heads
        dim_v = dim_out // heads

# (Page 5) BN after Q and V are helpful
        get_q = nn.Sequential(
            nn.Conv2d(in_channels=dim, out_channels=dim_k*heads, kernel_size=1, bias=False),
            nn.BatchNorm2d(dim_k*heads),
        )
        get_v = nn.Sequential(
            nn.Conv2d(in_channels=dim, out_channels=dim_v, kernel_size=1, bias=False),
            nn.BatchNorm2d(dim_v),
        )
        self.get_k = nn.Conv2d(in_channels=dim, out_channels=dim_k*heads, kernel_size=1, bias=False)

        ## TODO： 
        # embedding = ()
        # self.relative_position = generate_relative_positions_matrix(dim,dim)

    def forward(self,x):
        (b, c, im_h, im_w) = x.shape
        Q = self.get_q(x)
        K = self.get_k(x)
        V = self.get_v(x)
        
        Q = rearrange(Q, 'b (h k) im_h im_w -> b h k (im_h im_w)', h=self.heads)
        print(Q.size())
        K = rearrange(K, 'b k im_h im_w -> b k (im_h im_w)')
        print(K.size())
        V = rearrange(V, 'b v im_h im_w -> b v (im_h im_w)')
        print(V.size())
        σ_K = K.softmax(dim=-1)
        λc = einsum('b k m, b v m -> b k v', σ_K, V)
        # λp = einsum('b ') ## TODO: 添加Embedding

        # λn = λc + λp
        return λc

In [27]:
import torch
import torch.nn as nn
from einops import rearrange
from torch import einsum
from torchinfo import summary

# summary(LambdaLayer(dim=3,dim_k=3,dim_out=3,heads=3), (32,3,255,255))

X = torch.rand((32,3,255,255))
LambdaLayer(dim=3,dim_k=3,dim_out=3,heads=3)(X).size()


torch.Size([32, 3, 3, 65025])
torch.Size([32, 9, 65025])
torch.Size([32, 1, 65025])


torch.Size([32, 9, 1])

In [5]:
import torch 
import torch.nn as nn
def rel_pos_indices(size):
    size = (size, size)
    pos = torch.stack(torch.meshgrid(torch.arange(size[0]), torch.arange(size[1]))).flatten(1)
    rel_pos = pos[:, None, :] - pos[:, :, None]
    rel_pos[0] += size[0] - 1
    rel_pos[1] += size[1] - 1
    return rel_pos  # 2, H * W, H * W

def get_relative_position_matrix(size,):
    x = torch.repeat_interleave(torch.arange(size),size,dim=0)
    y = torch.arange(size).repeat(size)
    distance_mat = torch.stack((x,y))
    distance_mat = distance_mat[:,None, :] - distance_mat[:, :,None]
    distance_mat = torch.clamp(distance_mat, -size, size)
    distance_mat += size-1
    return distance_mat

rel_pos = get_relative_position_matrix(64)
rel_pos_2 = rel_pos_indices(64)
rel_lengths = 2 * 64 - 1 # n = im_h = im_w the feature map size
rel_pos_emb = nn.Parameter(torch.randn(rel_lengths, rel_lengths, 16)) # 2*n-1 2*n-1 k

torch.equal(rel_pos_emb[rel_pos[0],rel_pos[1]], rel_pos_emb[rel_pos_2[0],rel_pos_2[1]])


True

In [41]:
import torch 
def rel_pos_indices(size):
    size = (size, size)
    pos = torch.stack(torch.meshgrid(torch.arange(size[0]), torch.arange(size[1]))).flatten(1)
    rel_pos = pos[:, None, :] - pos[:, :, None]
    rel_pos[0] += size[0] - 1
    rel_pos[1] += size[1] - 1
    return rel_pos  # 2, H * W, H * W

  
from einops import rearrange
def get_relative_position_matrix(size,):
  x = torch.repeat_interleave(torch.arange(size),size,dim=0)
  y = torch.arange(size).repeat(size)
  distance_mat = torch.stack((x,y))
  distance_mat = distance_mat[:,None, :] - distance_mat[:, :,None]
  distance_mat = torch.clamp(distance_mat, -size, size)
  distance_mat += size-1
  return distance_mat

def calc_rel_pos(n):
    pos = torch.meshgrid(torch.arange(n), torch.arange(n))
    pos = rearrange(torch.stack(pos), 'n i j -> (i j) n')  # [n*n, 2] pos[n] = (i, j)
    rel_pos = pos[None, :] - pos[:, None]                  # [n*n, n*n, 2] rel_pos[n, m] = (rel_i, rel_j)
    rel_pos += n - 1                                       # shift value range from [-n+1, n-1] to [0, 2n-2]
    return rel_pos

print(get_relative_position_matrix(2))
print(rel_pos_indices(2))
print(calc_rel_pos(2))
torch.equal(get_relative_position_matrix(10), rel_pos_indices(10))

tensor([[[1, 1, 2, 2],
         [1, 1, 2, 2],
         [0, 0, 1, 1],
         [0, 0, 1, 1]],

        [[1, 2, 1, 2],
         [0, 1, 0, 1],
         [1, 2, 1, 2],
         [0, 1, 0, 1]]])
tensor([[[1, 1, 2, 2],
         [1, 1, 2, 2],
         [0, 0, 1, 1],
         [0, 0, 1, 1]],

        [[1, 2, 1, 2],
         [0, 1, 0, 1],
         [1, 2, 1, 2],
         [0, 1, 0, 1]]])
tensor([[[1, 1],
         [1, 2],
         [2, 1],
         [2, 2]],

        [[1, 0],
         [1, 1],
         [2, 0],
         [2, 1]],

        [[0, 1],
         [0, 2],
         [1, 1],
         [1, 2]],

        [[0, 0],
         [0, 1],
         [1, 0],
         [1, 1]]])


True

In [39]:
size=2

x = torch.repeat_interleave(torch.arange(size),size,dim=0)
y = torch.arange(size).repeat(size)
distance_mat = torch.stack((x,y))
distance_mat = distance_mat[:,None, :] - distance_mat[:, :,None]
distance_mat = torch.clamp(distance_mat, -size, size)
# distance_mat = rearrange(distance_mat, 'n i j -> (i j) n')
distance_mat += size-1
distance_mat

tensor([[[1, 1, 2, 2],
         [1, 1, 2, 2],
         [0, 0, 1, 1],
         [0, 0, 1, 1]],

        [[1, 2, 1, 2],
         [0, 1, 0, 1],
         [1, 2, 1, 2],
         [0, 1, 0, 1]]])

In [37]:
import torch
import torch.nn as nn
from einops import rearrange
from torch import einsum

def setup_seed(seed):
     torch.manual_seed(seed)
     torch.cuda.manual_seed_all(seed)
    #  np.random.seed(seed)
    #  random.seed(seed)
     torch.backends.cudnn.deterministic = True
# 设置随机数种子
setup_seed(20)

def calc_rel_pos(n):
    pos = torch.meshgrid(torch.arange(n), torch.arange(n))
    pos = rearrange(torch.stack(pos), 'n i j -> (i j) n')  # [n*n, 2] pos[n] = (i, j)
    rel_pos = pos[None, :] - pos[:, None]                  # [n*n, n*n, 2] rel_pos[n, m] = (rel_i, rel_j)
    rel_pos += n - 1                                       # shift value range from [-n+1, n-1] to [0, 2n-2]
    return rel_pos

def calc_embeddings(dim_k, n):
    rel_lengths = 2 * n - 1
    # rel_pos_emb = nn.Parameter(torch.randn(rel_lengths, rel_lengths, dim_k))
    rel_pos_emb = nn.Parameter(torch.arange(rel_lengths*rel_lengths*dim_k,dtype=float).reshape(rel_lengths,rel_lengths,dim_k))
    rel_pos = calc_rel_pos(n)

    n, m = rel_pos.unbind(dim = -1)
    rel_pos_emb = rel_pos_emb[n, m]
    return rel_pos_emb

def get_relative_position_matrix(size,):
  x = torch.repeat_interleave(torch.arange(size),size,dim=0)
  y = torch.arange(size).repeat(size)
  distance_mat = torch.stack((x,y))
  distance_mat = distance_mat[:,None, :] - distance_mat[:, :,None]
  distance_mat = torch.clamp(distance_mat, -size, size)
  distance_mat += size-1
  return distance_mat

def get_embedding(dim_k=1, n=2):
  rel_lengths = 2 * n - 1 # n = im_h = im_w the feature map size
  # rel_pos_emb = nn.Parameter(torch.randn(rel_lengths, rel_lengths, dim_k)) # 2*n-1 2*n-1 k 
  rel_pos_emb = nn.Parameter(torch.arange(rel_lengths*rel_lengths*dim_k,dtype=float).reshape(rel_lengths,rel_lengths,dim_k))
  rel_pos = get_relative_position_matrix(n)
  return rel_pos_emb[rel_pos[0], rel_pos[1]]

# print(calc_embeddings(1,10))
# print(get_embedding(1,10))
# torch.equal(calc_embeddings(1,10), get_embedding(1,10))
print(get_embedding(1,10).shape)

torch.Size([100, 100, 1])


In [8]:
from modeling.ResNet import LambdaLayer
from lambda_networks import LambdaLayer

layer1 = LambdaLayer(
    dim = 32,       # channels going in
    dim_out = 32,   # channels out
    n = 64,         # size of the receptive window - max(height, width)
    dim_k = 16,     # key dimension
    heads = 4,      # number of heads, for multi-query
    dim_u = 1       # 'intra-depth' dimension
)

#  dim, dim_k, n, dim_out, heads, r=None
layer2 = LambdaLayer(
    dim = 32,       # channels going in
    dim_k = 16,     # key dimension
    n = 64,         # size of the receptive window - max(height, width)
    dim_out = 32,   # channels out
    heads = 4,      # number of heads, for multi-query
    # r = 23,
)

x = torch.randn(1, 32, 64, 64)
print(layer1(x).shape)
print(layer2(x).shape)

torch.Size([1, 32, 64, 64])
torch.Size([1, 32, 64, 64])


In [6]:
import torch.nn as nn
def get_relative_position_matrix(size,):
    x = torch.repeat_interleave(torch.arange(size),size,dim=0)
    y = torch.arange(size).repeat(size)
    distance_mat = torch.stack((x,y))
    distance_mat = distance_mat[:,None, :] - distance_mat[:, :,None]
    distance_mat = torch.clamp(distance_mat, -size, size)
    distance_mat += size-1
    return distance_mat

def get_embedding(dim_k, n):
    rel_lengths = 2 * n - 1 # n = im_h = im_w the feature map size
    rel_pos_emb = nn.Parameter(torch.randn(rel_lengths, rel_lengths, dim_k)) # 2*n-1 2*n-1 k 
    rel_pos = get_relative_position_matrix(n)
    # return rel_pos
    return rel_pos_emb[rel_pos[0], rel_pos[1]]


n, m = get_embedding(16,64).unbind(dim = -1)

ValueError: too many values to unpack (expected 2)

In [10]:
from einops import rearrange
from torch import einsum
import torch.nn as nn
def get_relative_position_matrix(size,):
    x = torch.repeat_interleave(torch.arange(size),size,dim=0)
    y = torch.arange(size).repeat(size)
    distance_mat = torch.stack((x,y))
    distance_mat = distance_mat[:,None, :] - distance_mat[:, :,None]
    distance_mat = torch.clamp(distance_mat, -size, size)
    distance_mat += size-1
    return distance_mat

def get_embedding(dim_k, n):
    rel_lengths = 2 * n - 1 # n = im_h = im_w the feature map size
    rel_pos_emb = nn.Parameter(torch.randn(rel_lengths, rel_lengths, dim_k)) # 2*n-1 2*n-1 k 
    rel_pos = get_relative_position_matrix(n)
    # return rel_pos
    return rel_pos_emb[rel_pos[0], rel_pos[1]]

V = torch.randn(1, 64*64, 32//4)
embeddings = get_embedding(16, 64) # n m k
λp = einsum('n m k, b m v -> b n k v', embeddings, V)


torch.Size([1, 4096, 16, 8])


In [3]:
import torch.nn as nn 
import torch 
x = torch.rand(1,3,224,224)

conv1 = nn.Sequential(
    nn.Conv2d(3, 64, kernel_size=3, stride=1, padding=1,
                        bias=False),
    nn.Conv2d(3, 64, kernel_size=3, stride=1, padding=1,
                    bias=False),                  
)

bn1 = nn.BatchNorm2d(64)
relu = nn.ReLU(inplace=True)
maxpool = nn.MaxPool2d(kernel_size=3, stride=2, padding=1)

x = conv1(x)
x = bn1(x)
x = relu(x)
x = maxpool(x)
x.shape

RuntimeError: Given groups=1, weight of size [64, 3, 3, 3], expected input[1, 64, 224, 224] to have 3 channels, but got 64 channels instead

In [4]:
from modeling.ResNet import resnet18
import torch
model = resnet18()
x = torch.rand(1,3,224,224)
model(x)

torch.Size([3136, 3136, 16])
torch.Size([1, 3136, 16])
torch.Size([3136, 3136, 16])
torch.Size([1, 3136, 16])


tensor([[ 0.9174, -0.0215, -0.8486, -0.0110, -0.0028, -0.7266,  0.8609, -0.4256,
          0.7112,  0.7506]], grad_fn=<AddmmBackward0>)

In [10]:
import torch
import torch.nn as nn
from einops import rearrange, repeat

from torch import einsum

def get_relative_position_matrix(size,):
    x = torch.repeat_interleave(torch.arange(size),size,dim=0)
    y = torch.arange(size).repeat(size)
    distance_mat = torch.stack((x,y))
    distance_mat = distance_mat[:,None, :] - distance_mat[:, :,None]
    distance_mat = torch.clamp(distance_mat, -size, size)
    distance_mat += size-1
    return distance_mat

k=16
r=23
b=1
im_h=im_w=224
n = 56
dim_v=56//2

rel_pos = get_relative_position_matrix(n)
rel_lengths = 2 * n - 1 # n = im_h = im_w the feature map size
rel_pos_emb = nn.Parameter(torch.randn(rel_lengths, rel_lengths, k)) # 2*n-1 2*n-1 k 

Q = torch.rand(1,4,224*224,16)
V = torch.rand(1,56*56,dim_v)
# pos_conv = nn.Conv3d(1, k, (r, r, 1), padding=(r // 2, r // 2, 0))
# V = rearrange(V, 'b (im_h im_w) v -> b v im_h im_w', im_h = im_h, im_w = im_w).reshape(b, 1, dim_v, im_h, im_w,)
# embeddings = pos_conv(V)
# # embeddings.flatten(3).shape
# # Q.shape
# position_output = einsum('b h n k, b k v n -> b n h v', Q, embeddings.flatten(3))
# position_output.shape

# 56x56=3136
embeddings = rel_pos_emb[rel_pos[0], rel_pos[1]]
print(embeddings.shape)
print(V.shape)
λp = einsum('n m k, b m v -> b n k v', embeddings, V)

torch.Size([3136, 3136, 16])
torch.Size([1, 3136, 28])


In [23]:
import numpy as np 
y = np.array([5,7,10,6,9])
y_m = np.array([2,3,-4,3])
def AR(y):
    y_process=np.lib.stride_tricks.sliding_window_view(y,(2))
    output = []
    for y in y_process:
        y = y[::-1]
        output.append(0.8*y[0] + 0.3*y[1])


AR(y_m)

[3 2]
[-4  3]
[ 3 -4]


In [3]:
import pandas as pd 
csv_file = '/Volumes/Loading/datasets/imagewoof2/noisy_imagewoof.csv'
image_path_label = pd.read_csv(csv_file)
train_path_label = image_path_label.loc[image_path_label['is_valid'] == 0]
val_path_label = image_path_label.loc[image_path_label['is_valid'] == 1]

train_path_label.to_csv('train_noisy_imagewoof.csv')
val_path_label.to_csv('val_noisy_imagewoof.csv')

In [69]:
import torch

from torchvision import transforms
from torch.utils.data import Dataset, DataLoader
import numpy as np

import os
from PIL import Image
import pandas as pd 
from torchvision import transforms

image_root = '/Volumes/Loading/imagenette2'
csv_file = '/Volumes/Loading/imagenette2/noisy_imagenette.csv'

image_path_label = pd.read_csv(csv_file)
train_path_label = image_path_label.loc[image_path_label['is_valid'] == 0]
val_path_label = image_path_label.loc[image_path_label['is_valid'] == 1]

transform = transforms.Compose(
        [
            transforms.Resize((224, 224)),
            transforms.RandomHorizontalFlip(p=0.5),
            transforms.RandomRotation(15),
            transforms.ToTensor(),
            transforms.RandomErasing(0.2),
            transforms.Normalize((0.485, 0.456, 0.406), (0.229, 0.224, 0.225)),
        ]
    )



class ImageNette(Dataset):

    def __init__(self, csv_file, root_dir, noisy_level=0, transform=None):
        """
        Args:
            csv_file (string): Path to the csv file with annotations.
            root_dir (string): Directory with all the images.
            transform (callable, optional): Optional transform to be applied
                on a sample.
        """
        self.image_path_list = csv_file
        self.root_dir = root_dir
        self.transform = transform
        self.noisy_level = noisy_level
        self.train_map = {
            'n02979186': 0,
            'n03417042': 1,
            'n01440764': 2,
            'n02102040': 3,
            'n03028079': 4,
            'n03888257': 5,
            'n03394916': 6,
            'n03000684': 7,
            'n03445777': 8,
            'n03425413': 9
        }
        self.val_map = {
            'n02979186': 0,
            'n03417042': 1,
            'n01440764': 2,
            'n02102040': 3,
            'n03028079': 4,
            'n03888257': 5,
            'n03394916': 6,
            'n03000684': 7,
            'n03445777': 8,
            'n03425413': 9
        }


    def __len__(self):
        return len(self.image_path_list)

    def __getitem__(self, idx):
        if torch.is_tensor(idx):
            idx = idx.tolist()

        img_path = os.path.join(self.root_dir,
                                self.image_path_list['path'][idx])
        image = Image.open(img_path)
        labels_head = 'noisy_labels_'+str(self.noisy_level)
        label = self.val_map[self.image_path_list[labels_head][idx]]
        label = torch.from_numpy(np.array([label]))

        if self.transform:
            image = self.transform(image)
        sample =  [image, label]
        return sample

transformed_dataset = ImageNette(
    csv_file=train_path_label,
    root_dir=image_root,
    noisy_level=0,
    transform=transform,
)

train_dataloader = torch.utils.data.DataLoader(
    transformed_dataset, batch_size=6
)
from tqdm import tqdm
tk = tqdm(train_dataloader)
for t, data in enumerate(tk):
    print(data[0].shape)
    print(data[1].flatten())
    break
# for i in range(len(transformed_dataset)):
#     sample = transformed_dataset[i]

#     print(i, sample[0].size(), sample[1].size())

#     if i == 3:
#         break
# index = F.one_hot(torch.arange(0, 10), num_classes=10).tolist()

  0%|          | 0/1579 [00:00<?, ?it/s]

torch.Size([6, 3, 224, 224])
tensor([0, 0, 0, 0, 0, 0])


In [80]:
import torch.nn.functional as F

def softXEnt (input, target):
    logprobs = torch.nn.functional.log_softmax (input, dim = 1)
    return  -(target * logprobs).sum() / input.shape[0]

x = torch.randn(10,3)
y = torch.eye(10,3)

print(torch.nn.functional.cross_entropy(x, y))
softXEnt(x,y)   

tensor(0.3588)


tensor(0.3588)

In [6]:
import pandas as pd 
import numpy as np 
from PIL import Image
# csv_file = '/Volumes/Loading/imagenette2/noisy_imagenette.csv'
csv_file = '/Volumes/Loading/datasets/imagewoof2/noisy_imagewoof.csv'
image_root = '/Volumes/Loading/imagewoof2'
df = pd.read_csv(csv_file)
# idx = 1
# im_path = os.path.join(image_root, df['path'][idx])
# Image.open(im_path)
labels = list(df.loc[df['is_valid'] == 1]['noisy_labels_0'].unique())
index = np.arange(0, len(labels))
dict(zip(labels, index))

{'n02115641': 0,
 'n02086240': 1,
 'n02088364': 2,
 'n02087394': 3,
 'n02105641': 4,
 'n02111889': 5,
 'n02099601': 6,
 'n02096294': 7,
 'n02093754': 8,
 'n02089973': 9}

In [19]:
import torch
import torch.nn as nn
from einops import rearrange, repeat
# from lambda_networks import LambdaLayer
from torch import einsum
# make required imports
import torch 
import timm 
import os
import torchvision
import numpy as np 
import pandas as pd
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
from PIL import Image 
from matplotlib import pyplot as plt
from pathlib import Path 
from tqdm import tqdm
from sklearn.preprocessing import LabelEncoder
from dataclasses import dataclass
from timm.models.registry import register_model
from timm.models.helpers import build_model_with_cfg
from timm.models.resnet import Bottleneck, _create_resnet, default_cfgs, _cfg, make_blocks, create_classifier
import torch
import torch.nn as nn
from einops import rearrange, repeat
# from lambda_networks import LambdaLayer
from torch import einsum

# https://github.com/tensorflow/tensor2tensor/blob/master/tensor2tensor/layers/common_attention.py
def get_relative_position_matrix(size,):
    x = torch.repeat_interleave(torch.arange(size),size,dim=0)
    y = torch.arange(size).repeat(size)
    distance_mat = torch.stack((x,y))
    distance_mat = distance_mat[:,None, :] - distance_mat[:, :,None]
    distance_mat = torch.clamp(distance_mat, -size, size)
    distance_mat += size-1
    return distance_mat

class LambdaLayer_own(nn.Module):
    def __init__(self, dim, dim_k=16,n=64, dim_out=None, heads=4, r=None):
        super().__init__()
        self.dim_out = dim_out
        self.r = r
        self.n = n
        self.k = dim_k
        assert(dim_out % heads) == 0
        self.heads = heads
        self.dim_v = dim_out // heads

# (Page 5) BN after Q and V are helpful
        self.get_q = nn.Sequential(
            nn.Conv2d(in_channels=dim, out_channels=dim_k*heads, kernel_size=1, bias=False),
            nn.BatchNorm2d(dim_k*heads),
        )
        self.get_v = nn.Sequential(
            nn.Conv2d(in_channels=dim, out_channels=self.dim_v, kernel_size=1, bias=False),
            nn.BatchNorm2d(self.dim_v),
        )
        self.get_k = nn.Conv2d(in_channels=dim, out_channels=dim_k, kernel_size=1, bias=False)

        if r is not None:
            assert (r % 2) == 1, 'Receptive kernel size should be odd'
            self.pos_conv = nn.Conv3d(1, self.k, (r, r, 1), padding=(r // 2, r // 2, 0))
        else:   
            # rel_lengths = 2 * n - 1 # n = im_h = im_w the feature map size
            # self.rel_pos_emb = nn.Parameter(torch.randn(rel_lengths, rel_lengths, dim_k)) # n m k 
            self.rel_pos = get_relative_position_matrix(n)
            rel_lengths = 2 * self.n - 1 # n = im_h = im_w the feature map size
            self.rel_pos_emb = nn.Parameter(torch.randn(rel_lengths, rel_lengths, self.k)) # 2*n-1 2*n-1 k 

    def forward(self,x):
        (b, c, im_h, im_w) = x.shape
        Q = self.get_q(x)
        K = self.get_k(x)
        V = self.get_v(x)

        # m = n = im_h * im_w
        Q = rearrange(Q, 'b (h k) im_h im_w -> b h (im_h im_w) k', h=self.heads) # b h n(m) k 
        K = rearrange(K, 'b k im_h im_w -> b (im_h im_w) k') # b m k 
        V = rearrange(V, 'b v im_h im_w -> b (im_h im_w) v') # b m v

        σ_K = K.softmax(dim=-1)
        λc = einsum('b m k, b m v -> b k v', σ_K, V)
        content_output = einsum('b h n k, b k v -> b n h v', Q, λc)

        if self.r is not None:
            V = rearrange(V, 'b (im_h im_w) v -> b v im_h im_w', im_h = im_h, im_w = im_w).reshape(b, 1, self.dim_v, im_h, im_w,)
            embeddings = self.pos_conv(V)
            position_output = einsum('b h n k, b k v n -> b n h v', Q, embeddings.flatten(3))
        else:
            # embeddings = get_embedding(self.k, im_h) # n m k
            embeddings = self.rel_pos_emb[self.rel_pos[0], self.rel_pos[1]]
#             print(embeddings.shape)
#             print(V.shape)
            λp = einsum('n m k, b m v -> b n k v', embeddings, V)
            position_output = einsum('b h n k, b n k v -> b n h v',Q, λp)

        output = rearrange(content_output + position_output, 'b (im_h im_w) h v -> b (h v) im_h im_w', im_h=im_h, im_w=im_w)

        return output


def conv3x3(in_planes, out_planes, stride=1, groups=1, dilation=1):
    """3x3 convolution with padding"""
    return nn.Conv2d(in_planes, out_planes, kernel_size=3, stride=stride,
                     padding=dilation, groups=groups, bias=False, dilation=dilation)


def conv1x1(in_planes, out_planes, stride=1):
    """1x1 convolution"""
    return nn.Conv2d(in_planes, out_planes, kernel_size=1, stride=stride, bias=False)


class BasicBlock(nn.Module):
    expansion = 1

    def __init__(self, inplanes, planes, stride=1, downsample=None, groups=1,
                 base_width=64, dilation=1, norm_layer=None,feature_size=None):
        super(BasicBlock, self).__init__()
        if norm_layer is None:
            norm_layer = nn.BatchNorm2d
        if groups != 1 or base_width != 64:
            raise ValueError('BasicBlock only supports groups=1 and base_width=64')
        if dilation > 1:
            raise NotImplementedError("Dilation > 1 not supported in BasicBlock")
        # Both self.conv1 and self.downsample layers downsample the input when stride != 1
        self.conv1 = conv3x3(inplanes, planes, stride)
        self.bn1 = norm_layer(planes)
        self.relu = nn.ReLU(inplace=True)
        # self.conv2 = conv3x3(planes, planes)
#         self.conv2 = LambdaLayer(
#                             dim = planes,
#                             dim_out = planes,
#                             n = feature_size,
# #                             r = 23,         # the receptive field for relative positional encoding (23 x 23)
#                             dim_k = 16,
#                             heads = 4,
#                             dim_u = 4
#                         )
        self.conv2 = LambdaLayer_own(dim=planes, dim_out=planes, n=feature_size)
        self.bn2 = norm_layer(planes)
        self.downsample = downsample
        self.stride = stride

    def forward(self, x):
        identity = x

        out = self.conv1(x)
        out = self.bn1(out)
        out = self.relu(out)

        out = self.conv2(out)
        out = self.bn2(out)

        if self.downsample is not None:
            identity = self.downsample(x)

        out += identity
        out = self.relu(out)

        return out

class BasicBlock_Rs(nn.Module):
    expansion = 1

    def __init__(self, inplanes, planes, stride=1, downsample=None, groups=1,
                 base_width=64, dilation=1, norm_layer=None,feature_size=None):
        super(BasicBlock_Rs, self).__init__()
        if norm_layer is None:
            norm_layer = nn.BatchNorm2d
        if groups != 1 or base_width != 64:
            raise ValueError('BasicBlock only supports groups=1 and base_width=64')
        if dilation > 1:
            raise NotImplementedError("Dilation > 1 not supported in BasicBlock")
        # Both self.conv1 and self.downsample layers downsample the input when stride != 1
        self.conv1 = conv3x3(inplanes, planes, stride)
        self.bn1 = norm_layer(planes)
        self.relu = nn.ReLU(inplace=True)
        self.conv2 = conv3x3(planes, planes)
        # self.conv2 = LambdaLayer(dim=planes, dim_out=planes)
        self.bn2 = norm_layer(planes)
        self.downsample = downsample
        self.stride = stride

    def forward(self, x):
        identity = x

        out = self.conv1(x)
        out = self.bn1(out)
        out = self.relu(out)

        out = self.conv2(out)
        out = self.bn2(out)

        if self.downsample is not None:
            identity = self.downsample(x)

        out += identity
        out = self.relu(out)

        return out

    

class ResNet(nn.Module):

    def __init__(self, block_lambda, block_rs, layers, num_classes=10, zero_init_residual=False,
                 groups=1, width_per_group=64, replace_stride_with_dilation=None,
                 norm_layer=None,):
        super(ResNet, self).__init__()
        if norm_layer is None:
            norm_layer = nn.BatchNorm2d
        self._norm_layer = norm_layer

        self.inplanes = 64
        self.dilation = 1
        if replace_stride_with_dilation is None:
            # each element in the tuple indicates if we should replace
            # the 2x2 stride with a dilated convolution instead
            replace_stride_with_dilation = [False, False, False]
        if len(replace_stride_with_dilation) != 3:
            raise ValueError("replace_stride_with_dilation should be None "
                             "or a 3-element tuple, got {}".format(replace_stride_with_dilation))
        self.groups = groups
        self.base_width = width_per_group
        # self.conv1 = nn.Conv2d(3, self.inplanes, kernel_size=3, stride=1, padding=1,
        #                        bias=False)
        self.conv1 = nn.Conv2d(3, self.inplanes, kernel_size=7, stride=2, padding=3,
                               bias=False)
        self.bn1 = norm_layer(self.inplanes)
        self.relu = nn.ReLU(inplace=True)
        self.maxpool = nn.MaxPool2d(kernel_size=3, stride=2, padding=1)
        self.layer1 = self._make_layer(block_rs, 64, layers[0], feature_size=56)
        self.layer2 = self._make_layer(block_rs, 128, layers[1], stride=2,
                                       dilate=replace_stride_with_dilation[0],feature_size=28) # 56/2=
        self.layer3 = self._make_layer(block_lambda, 256, layers[2], stride=2,
                                       dilate=replace_stride_with_dilation[1],feature_size=14) # 14
        self.layer4 = self._make_layer(block_rs, 512, layers[3], stride=2,
                                       dilate=replace_stride_with_dilation[2],feature_size=7) # 7
        self.avgpool = nn.AdaptiveAvgPool2d((1, 1))
        self.fc = nn.Linear(512 * block_rs.expansion, num_classes)

        for m in self.modules():
            if isinstance(m, nn.Conv2d):
                nn.init.kaiming_normal_(m.weight, mode='fan_out', nonlinearity='relu')
            elif isinstance(m, (nn.BatchNorm2d, nn.GroupNorm)):
                nn.init.constant_(m.weight, 1)
                nn.init.constant_(m.bias, 0)

        # Zero-initialize the last BN in each residual branch,
        # so that the residual branch starts with zeros, and each residual block behaves like an identity.
        # This improves the model by 0.2~0.3% according to https://arxiv.org/abs/1706.02677
        if zero_init_residual:
            for m in self.modules():
                if isinstance(m, Bottleneck):
                    nn.init.constant_(m.bn3.weight, 0)
                elif isinstance(m, BasicBlock):
                    nn.init.constant_(m.bn2.weight, 0)

    def _make_layer(self, block, planes, blocks, stride=1, dilate=False, feature_size=None):
        norm_layer = self._norm_layer
        downsample = None
        previous_dilation = self.dilation
        if dilate:
            self.dilation *= stride
            stride = 1
        if stride != 1 or self.inplanes != planes * block.expansion:
            downsample = nn.Sequential(
                conv1x1(self.inplanes, planes * block.expansion, stride),
                norm_layer(planes * block.expansion),
            )

        layers = []
        layers.append(block(self.inplanes, planes, stride, downsample, self.groups,
                            self.base_width, previous_dilation, norm_layer, feature_size=feature_size))
        self.inplanes = planes * block.expansion
        for _ in range(1, blocks):

            layers.append(block(self.inplanes, planes, groups=self.groups,
                                base_width=self.base_width, dilation=self.dilation,
                                norm_layer=norm_layer,feature_size=feature_size))


        return nn.Sequential(*layers)

    def _forward_impl(self, x):
        # See note [TorchScript super()]
        x = self.conv1(x)
        x = self.bn1(x)
        x = self.relu(x)
        x = self.maxpool(x)

        x = self.layer1(x)
        x = self.layer2(x)
        x = self.layer3(x)
        x = self.layer4(x)

        x = self.avgpool(x)
        x = torch.flatten(x, 1)
        x = self.fc(x)

        return x

    def forward(self, x):
        return self._forward_impl(x)


def _resnet(arch, block,BasicBlock_Rs, layers, pretrained, progress, **kwargs):
    model = ResNet(block, BasicBlock_Rs, layers, **kwargs)
    # if pretrained:
    #     state_dict = load_state_dict_from_url(model_urls[arch],
    #                                           progress=progress)
    #     model.load_state_dict(state_dict)
    return model


def resnet18(pretrained=False, progress=True, **kwargs):
    r"""ResNet-18 model from
    `"Deep Residual Learning for Image Recognition" <https://arxiv.org/pdf/1512.03385.pdf>`_

    Args:
        pretrained (bool): If True, returns a model pre-trained on ImageNet
        progress (bool): If True, displays a progress bar of the download to stderr
    """
    return _resnet('resnet18', BasicBlock,BasicBlock_Rs, [2, 2, 2, 2], pretrained, progress,
                   **kwargs)

# [2, 2, 2, 2] RL-26
# [2, 3, 5, 2] RL-38
# [3, 4, 6, 3] RL-50

def resnet34(pretrained=False, progress=True, **kwargs):
    r"""ResNet-34 model from
    `"Deep Residual Learning for Image Recognition" <https://arxiv.org/pdf/1512.03385.pdf>`_

    Args:
        pretrained (bool): If True, returns a model pre-trained on ImageNet
        progress (bool): If True, displays a progress bar of the download to stderr
    """
    return _resnet('resnet34', BasicBlock,BasicBlock_Rs, [2, 3, 5, 2], pretrained, progress,
                   **kwargs)



def resnet50(pretrained=False, progress=True, **kwargs):
    r"""ResNet-50 model from
    `"Deep Residual Learning for Image Recognition" <https://arxiv.org/pdf/1512.03385.pdf>`_

    Args:
        pretrained (bool): If True, returns a model pre-trained on ImageNet
        progress (bool): If True, displays a progress bar of the download to stderr
    """
    return _resnet('resnet50', BasicBlock, BasicBlock_Rs,[3, 4, 6, 3], pretrained, progress,
                   **kwargs)

# L R (r None)2
18: 11012586  1.61GFlops
34: 14765498  2.43GFlops

# L R (r None) 3
18: 10099562 1.61GFlops
34: 12313882 2.43GFlops

# L R (r 23)2
18: 10932746  1.61GFlops
34: 14645738  2.43GFlops

# L R (r 23) 3
18: 10093194 1.61GFlops
34: 12297962 2.43GFlops

# res
18: 11689512
34: 21797672

woof noisy25 resnet34 
time: 3h 35m 37s
train_loss: 2.238
eval_loss: 2.295
woof noisy25 resnet18
time: 3h 7m 0s
train_loss: 2.232
eval_loss: 2.295

woof noisy0 resnet34
time: 2h 29m 54s
train_loss: 2.303
eval_loss: 2.282
woof noisy0 resnet18
time: 2h 6s
train_loss: 2.301
eval_loss: 2.287

imagenette noisy5 resnet34
time: 2h 21m 3s
train_loss: 2.287
eval_loss: 2.32

imagenette noisy5 resnet18
time: 1h 58m 53s
train_loss: 2.307
eval_loss: 2.317

ImageNette
R18_0 
time: 1h 32m 37s
train_loss: 2.1614
eval_loss: 2.2887

R34_0
time: 1h 57m 2s
train_loss: 2.3295
eval_loss: 2.1858
R18_50
time: 1h 35m 42s
train_loss: 2.3360
eval_loss: 2.3483
R34_50
time: 1h 41m 4s
train_loss: 2.3372
eval_loss: 2.3442


1. imagenette-noisy25-resnet18
	runtime: 2h 40m 22s
	best train_loss: 2.332
	best eval_loss: 2.331

2. imagenette-noisy25-resnet34
	runtime: 1h 49m 59s
	best train_loss: 2.324
	best eval_loss: 2.33

3. imagenette-noisy25-resnet50
	runtime: 2h 23m 10s
	best train_loss: 2.181
	best eval_loss: 2.31

4. imagewoof-noisy5-resnet18
	runtime: 3h 11m 4s
	best train_loss: 2.332
	best eval_loss: 2.323

5. imagewoof-noisy5-resnet34
	runtime: 3h 21m 33
	best train_loss: 2.33
	best eval_loss: 2.318

6. imagewoof-noisy5-resnet50
	runtime: 3h 4m 26s
	best train_loss: 2.239
	best eval_loss: 2.288

7. imagewoof-noisy50-resnet18
	runtime: 2h 21m 37s
	best train_loss: 2.364
	best eval_loss: 2.333

8. imagewoof-noisy50-resnet34
	runtime: 3h 18m 59s
	best train_loss: 2.344
	best eval_loss: 2.326

9. imagewoof-noisy50-resnet50
	runtime: 2h 29m 16s
	best train_loss: 2.301
	best eval_loss: 2.298

	

In [21]:
from modeling.ResNet import resnet18, resnet34
import torch 
from torchstat import stat
from torchvision import models
# model = models.resnet34()
model = resnet34()
# model = resnet34()
# model.load_state_dict(torch.load('models/LambdaR18_0_lambda2_r_None.pt',map_location=torch.device('cpu')))
# # model = model.layer2[-1]
# model.eval()
stat(model, (3,224, 224))

[MAdd]: AdaptiveAvgPool2d is not supported!
[Flops]: AdaptiveAvgPool2d is not supported!
[Memory]: AdaptiveAvgPool2d is not supported!
[MAdd]: AdaptiveAvgPool2d is not supported!
[Flops]: AdaptiveAvgPool2d is not supported!
[Memory]: AdaptiveAvgPool2d is not supported!
[MAdd]: AdaptiveAvgPool2d is not supported!
[Flops]: AdaptiveAvgPool2d is not supported!
[Memory]: AdaptiveAvgPool2d is not supported!
[MAdd]: AdaptiveAvgPool2d is not supported!
[Flops]: AdaptiveAvgPool2d is not supported!
[Memory]: AdaptiveAvgPool2d is not supported!
[MAdd]: AdaptiveAvgPool2d is not supported!
[Flops]: AdaptiveAvgPool2d is not supported!
[Memory]: AdaptiveAvgPool2d is not supported!
[MAdd]: AdaptiveAvgPool2d is not supported!
[Flops]: AdaptiveAvgPool2d is not supported!
[Memory]: AdaptiveAvgPool2d is not supported!
[MAdd]: AdaptiveAvgPool2d is not supported!
[Flops]: AdaptiveAvgPool2d is not supported!
[Memory]: AdaptiveAvgPool2d is not supported!
[MAdd]: AdaptiveAvgPool2d is not supported!
[Flops]: Ad

/Users/xuanli/miniforge3/envs/torch/lib/python3.8/site-packages/torchstat/reporter.py:63: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(total_df)


In [16]:
# L_R_18 = resnet18()
# L_R_34 = resnet34()

# print(sum([i.numel() for i in L_R_18.parameters()]))
# print(sum([i.numel() for i in L_R_34.parameters()]))

res18 = timm.create_model('resnet18',)
res34 = timm.create_model('resnet34',)

print(sum([i.numel() for i in res18.parameters()]))
print(sum([i.numel() for i in res34.parameters()]))

11689512
21797672


In [17]:
import argparse
import cv2
import numpy as np
import torch
from torchvision import models

from pytorch_grad_cam import GradCAM, \
                             ScoreCAM, \
                             GradCAMPlusPlus, \
                             AblationCAM, \
                             XGradCAM, \
                             EigenCAM, \
                             EigenGradCAM

from pytorch_grad_cam import GuidedBackpropReLUModel
from pytorch_grad_cam.utils.image import show_cam_on_image, \
                                         deprocess_image, \
                                         preprocess_image


# 如果出现 OMP: Error #15: Initializing libiomp5.dylib, but found libomp.dylib already initialized.
import os
os.environ["KMP_DUPLICATE_LIB_OK"]="TRUE"

def get_args():
    parser = argparse.ArgumentParser()
    parser.add_argument('--use-cuda', action='store_true', default=False,
                        help='Use NVIDIA GPU acceleration')
    parser.add_argument('--image-path', type=str, default='dog.JPEG',
                        help='Input image path')
    parser.add_argument('--aug_smooth', action='store_true',
                        help='Apply test time augmentation to smooth the CAM')
    parser.add_argument('--eigen_smooth', action='store_true',
                        help='Reduce noise by taking the first principle componenet'
                        'of cam_weights*activations')
    parser.add_argument('--method', type=str, default='gradcam',
                        choices=['gradcam', 'gradcam++', 'scorecam', 'xgradcam',
                                 'ablationcam', 'eigencam', 'eigengradcam'],
                        help='Can be gradcam/gradcam++/scorecam/xgradcam'
                             '/ablationcam/eigencam/eigengradcam')

    args = parser.parse_args()
    args.use_cuda = args.use_cuda and torch.cuda.is_available()
    if args.use_cuda:
        print('Using GPU for acceleration')
    else:
        print('Using CPU for computation')

    return args


if __name__ == '__main__':
    """ python cam.py -image-path <path_to_image>
    Example usage of loading an image, and computing:
        1. CAM
        2. Guided Back Propagation
        3. Combining both
    """

    args = get_args()
    methods = \
        {"gradcam": GradCAM,
         "scorecam": ScoreCAM,
         "gradcam++": GradCAMPlusPlus,
         "ablationcam": AblationCAM,
         "xgradcam": XGradCAM,
         "eigencam": EigenCAM,
         "eigengradcam": EigenGradCAM}

    model = timm.create_model('resnet18',pretrained=True)

    # Choose the target layer you want to compute the visualization for.
    # Usually this will be the last convolutional layer in the model.
    # Some common choices can be:
    # Resnet18 and 50: model.layer4[-1]
    # VGG, densenet161: model.features[-1]
    # mnasnet1_0: model.layers[-1]
    # You can print the model to help chose the layer
    target_layer = model.layer4[-1]

    cam = methods[args.method](model=model,
                               target_layer=target_layer,
                               use_cuda=args.use_cuda)

    rgb_img = cv2.imread(args.image_path, 1)[:, :, ::-1]
    rgb_img = np.float32(rgb_img) / 255
    input_tensor = preprocess_image(rgb_img, mean=[0.485, 0.456, 0.406],
                                             std=[0.229, 0.224, 0.225])

    # If None, returns the map for the highest scoring category.
    # Otherwise, targets the requested category.
    target_category = None

    # AblationCAM and ScoreCAM have batched implementations.
    # You can override the internal batch size for faster computation.
    cam.batch_size = 32

    grayscale_cam = cam(input_tensor=input_tensor,
                        target_category=target_category,
                        aug_smooth=args.aug_smooth,
                        eigen_smooth=args.eigen_smooth)

    # Here grayscale_cam has only one image in the batch
    grayscale_cam = grayscale_cam[0, :]

    cam_image = show_cam_on_image(rgb_img, grayscale_cam)

    gb_model = GuidedBackpropReLUModel(model=model, use_cuda=args.use_cuda)
    gb = gb_model(input_tensor, target_category=target_category)

    cam_mask = cv2.merge([grayscale_cam, grayscale_cam, grayscale_cam])
    cam_gb = deprocess_image(cam_mask * gb)
    gb = deprocess_image(gb)

    cv2.imwrite(f'{args.method}_cam.jpg', cam_image)
    cv2.imwrite(f'{args.method}_gb.jpg', gb)
    cv2.imwrite(f'{args.method}_cam_gb.jpg', cam_gb)

usage: ipykernel_launcher.py [-h] [--use-cuda] [--image-path IMAGE_PATH]
                             [--aug_smooth] [--eigen_smooth]
                             [--method {gradcam,gradcam++,scorecam,xgradcam,ablationcam,eigencam,eigengradcam}]
ipykernel_launcher.py: error: unrecognized arguments: -f /Users/xuanli/Library/Jupyter/runtime/kernel-25e1cd1e-8866-4777-84ce-6f3c05cf6e2e.json


SystemExit: 2

/Users/xuanli/miniforge3/envs/torch/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3465: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


In [9]:
from modeling.ResNet import resnet18, resnet34
import torch 
from torchstat import stat
from torchvision import models
model = models.resnet34()
# model = resnet18()
# model.load_state_dict(torch.load('models/LambdaR18_0_lambda2_r_None.pt',map_location=torch.device('cpu')))
# # model = model.layer2[-1]
# model.eval()
stat(model, (3,224, 224))

[MAdd]: AdaptiveAvgPool2d is not supported!
[Flops]: AdaptiveAvgPool2d is not supported!
[Memory]: AdaptiveAvgPool2d is not supported!
[MAdd]: AdaptiveAvgPool2d is not supported!
[Flops]: AdaptiveAvgPool2d is not supported!
[Memory]: AdaptiveAvgPool2d is not supported!
[MAdd]: AdaptiveAvgPool2d is not supported!
[Flops]: AdaptiveAvgPool2d is not supported!
[Memory]: AdaptiveAvgPool2d is not supported!
[MAdd]: AdaptiveAvgPool2d is not supported!
[Flops]: AdaptiveAvgPool2d is not supported!
[Memory]: AdaptiveAvgPool2d is not supported!
[MAdd]: AdaptiveAvgPool2d is not supported!
[Flops]: AdaptiveAvgPool2d is not supported!
[Memory]: AdaptiveAvgPool2d is not supported!
[MAdd]: AdaptiveAvgPool2d is not supported!
[Flops]: AdaptiveAvgPool2d is not supported!
[Memory]: AdaptiveAvgPool2d is not supported!
[MAdd]: AdaptiveAvgPool2d is not supported!
[Flops]: AdaptiveAvgPool2d is not supported!
[Memory]: AdaptiveAvgPool2d is not supported!
                 module name  input shape output shape 

/Users/xuanli/miniforge3/envs/torch/lib/python3.8/site-packages/torchstat/reporter.py:63: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(total_df)


In [67]:
model = timm.create_model('lambda_resnet50ts', pretrained=True)
model.final_conv


Downloading: "https://github.com/rwightman/pytorch-image-models/releases/download/v0.1-attn-weights/lambda_resnet50ts_a1h_256-b87370f7.pth" to /Users/xuanli/.cache/torch/hub/checkpoints/lambda_resnet50ts_a1h_256-b87370f7.pth


Identity()

In [7]:
import pyautogui as auto 
while(1): 
    auto.typewrite('A',interval=1)  
    auto.typewrite(["backspace"],interval=1)  


KeyboardInterrupt: 

In [19]:
import timm 
model = timm.create_model('resnet50')
# model.stages[-1][-1].conv3_1x1.bn
# model.layer4[-1]
model.layer2[-1]

Bottleneck(
  (conv1): Conv2d(512, 128, kernel_size=(1, 1), stride=(1, 1), bias=False)
  (bn1): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (act1): ReLU(inplace=True)
  (conv2): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
  (bn2): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (act2): ReLU(inplace=True)
  (conv3): Conv2d(128, 512, kernel_size=(1, 1), stride=(1, 1), bias=False)
  (bn3): BatchNorm2d(512, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (act3): ReLU(inplace=True)
)

In [1]:
from config import get_config

get_config('lambda18_l2_r_none')


'lambda18_l2_r_none'

In [2]:
from model import resnet18, resnet34
model = resnet34()
model

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock_Rs(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock_Rs(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=Tr